### Prompt:
 Identify a large 2-node network dataset—you can start with a dataset in a repository.  Your data should meet the criteria that it consists of ties between and not within two (or more) distinct groups.
    Reduce the size of the network using a method such as the island method described in chapter 4 of social network analysis.
    What can you infer about each of the distinct groups?

You may work in a small group on the project.

Your code and analysis should be delivered in an IPython Notebook by end of day Sunday. 

### Data
https://snap.stanford.edu/data/wiki-Elec.html

### Libraries

In [1]:
import pandas as pd
import os
import io
import networkx as nx
import re

importedScipy = False
try:    
    import scipy
    importedScipy = True
except:
    pass
try:    
    import nltk
except:
    pass

In [2]:
print ("Pandas Version {}".format(pd.__version__))
print ("Newtworkx Version {}".format(nx.__version__))
if (importedScipy):
    print ("scipy Version {}".format(scipy.__version__))
else:
    print("scipy wasn't imported.")
print("NLTK Version {}".format(nltk.__version__))

Pandas Version 1.3.5
Newtworkx Version 2.6.3
scipy Version 1.7.3
NLTK Version 3.7


### Read in Data

In [3]:
#The filename may have to be renamed with the .txt extention.  That has to be done manually.
#This was only the case on one of my machines for some reason??
filename = "wikiElec.ElecBs3.txt"

In [98]:
file = io.open(filename, mode="r", encoding="ANSI") #This encoding was missing before
print(file.read(1900))
data = file.readlines()

# Wikipedia elections (http://cs.stanford.edu/people/jure/pubs/triads-chi10.pdf). Data format:
#   E: is election succesful (1) or not (0)
#   T: time election was closed
#   U: user id (and username) of editor that is being considered for promotion
#   N: user id (and username) of the nominator
#   V: <vote(1:support, 0:neutral, -1:oppose)> <user_id> <time> <username>
E	1
T	2004-09-21 01:15:53
U	30	cjcurrie
N	32	andyl
V	1	3	2004-09-14 16:26:00	ludraman
V	-1	25	2004-09-14 16:53:00	blankfaze
V	1	4	2004-09-14 17:08:00	gzornenplatz
V	1	5	2004-09-14 17:37:00	orthogonal
V	1	6	2004-09-14 19:28:00	andrevan
V	1	7	2004-09-14 19:37:00	texture
V	1	8	2004-09-14 21:04:00	lst27
V	1	9	2004-09-14 21:30:00	mirv
V	1	10	2004-09-14 22:13:00	anÃ¡rion
V	0	26	2004-09-14 22:18:00	grunt
V	0	27	2004-09-15 03:19:00	slowking
V	0	28	2004-09-15 03:20:00	neutrality
V	1	11	2004-09-15 04:28:00	merovingian
V	1	12	2004-09-15 06:56:00	wile
V	1	13	2004-09-15 09:19:00	sjc
V	1	14	2004-09-15 12:20:00	172
V	0	29	2004-09-16 00

In [5]:
for line in data[:5]:
    print(line)

4	2004-09-14 17:08:00	gzornenplatz

V	1	5	2004-09-14 17:37:00	orthogonal

V	1	6	2004-09-14 19:28:00	andrevan

V	1	7	2004-09-14 19:37:00	texture

V	1	8	2004-09-14 21:04:00	lst27



In [6]:
data[0]

'4\t2004-09-14 17:08:00\tgzornenplatz\n'

In [7]:
data[1]

'V\t1\t5\t2004-09-14 17:37:00\torthogonal\n'

In [8]:
data[2]

'V\t1\t6\t2004-09-14 19:28:00\tandrevan\n'

I don't know why, but the reading in of the file starts at an abitrary spot and the first election is screwed up.  We'll just disregard it, I know from previous data exploration that are 2900+ other elections.  

### Discarding the first several screwed up rows

In [9]:
for i in range(len(data)):
    if data[i][0] =="E": #first proper "election in the data, as read in"
        print(i)
        break

26


In [10]:
data[26:35]

['E\t1\n',
 'T\t2004-09-21 20:49:47\n',
 'U\t54\tzoney\n',
 'N\t28\tneutrality\n',
 'V\t1\t28\t2004-09-14 13:39:00\tneutrality\n',
 'V\t1\t33\t2004-09-14 13:41:00\tchmod007\n',
 'V\t1\t34\t2004-09-14 14:40:00\tnorm\n',
 'V\t1\t5\t2004-09-14 15:00:00\torthogonal\n',
 'V\t1\t20\t2004-09-14 15:43:00\tmichael\n']

In [11]:
thisCellRan1 = False
# Do no rerun this cell or you will "erase" data from memory and need to rerun the notebook

In [12]:
if (thisCellRan1 == False):
    data = data[26:]
    thisCellRan1 = True  ##This first 25 cells have been discarded already
    del thisCellRan1
        
#dicarding the first 25 rows
#Only run this cell once!!!  That's the point of the thisCellRan1 variable.

In [13]:
data[0:5]
#line 0 should start with "E", election. 
#If it doesn't, restart the kernel and rerun the notebook.

['E\t1\n',
 'T\t2004-09-21 20:49:47\n',
 'U\t54\tzoney\n',
 'N\t28\tneutrality\n',
 'V\t1\t28\t2004-09-14 13:39:00\tneutrality\n']

In [14]:
ECount = 0
for i in data:
    if i[0] =="E":
        ECount +=1
ECount
#there should be 2793 elections in the dataset, as read in (the first elction read in wrong)

2793

### Class Definitions

to work with data

In [15]:
#from the docs
#E: is election succesful (1) or not (0)\\r\\n#   
#T: time election was closed\\r\\n#   
#U: user id (and username) of editor that is being considered for promotion\\r\\n#   
#N: user id (and username) of the nominator\\r\\n#
#V: <vote(1:support, 0:neutral, -1:oppose)> <user_id> <time> <username>

class Election():# A node type in the graph
    def __init__(self, success, time, user, nominator, votes):
        if((success!= 1) & (success!=0)):
            print("invalid success entered: ",success)
            self.success = "unknown"
        else:
            self.success = success
        self.time = 0 #TODO possibly if we want more edge data
        self.user = user
        self.nominator = nominator
        self.votes = votes

In [16]:
#   V: <vote(1:support, 0:neutral, -1:oppose)> <user_id> <time> <username>
class Vote: #Edges in the graph
    def __init__(self, vote, user_id, time, user_name):
        if ((vote <-1) | (vote >1)):
            print("invalid vote:", vote)
            self.vote = "invalid"
        else:
            self.vote = vote
        self.time = 0 #TODO if we want more attributes
        self.user_id = user_id
        self.user_name = user_name

#### Misc Testing

In [17]:
votes = []
for line in data[3:5]:
    if line[0]=="V":
        print(line)
        print(line[2]) #vote
        print(re.search("\d+", line[3:]))
    

V	1	28	2004-09-14 13:39:00	neutrality

1
<re.Match object; span=(1, 3), match='28'>


In [18]:
data[55]

'V\t1\t11\t2004-09-12 06:21:00\tmerovingian\n'

In [19]:
data[55][3:]

'\t11\t2004-09-12 06:21:00\tmerovingian\n'

In [20]:
match2 = re.search("\d+",data[55][3:]) 

In [21]:
findall2 = re.findall("\d+" , data[55])

In [22]:
findall2

['1', '11', '2004', '09', '12', '06', '21', '00']

In [23]:
findall3 = re.findall("[a-z]+", data[55])

In [24]:
findall3

['merovingian']

In [25]:
elections =[]
election =""
inAnElection = False
for i in range(len(data)):
    if ( (data[i][0] == "E") ): #&\
        #((i != len(data)-1) & (data[i+1][0]=="T") ) ) : #new election
        elections.append(election)
        election = data[i]
    else:
        election =  election + line

In [26]:
elections[3]

'E\t1\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1\t28\t2004-09-14 13:39:00\tneutrality\nV\t1

In [27]:
for i in range(len(data[:6])):
    print (data[i])                               

E	1

T	2004-09-21 20:49:47

U	54	zoney

N	28	neutrality

V	1	28	2004-09-14 13:39:00	neutrality

V	1	33	2004-09-14 13:41:00	chmod007



### Election Objects

In [28]:
electionsList =[]
for i in range(len(data)):
    #print (data[i])
    if ((data[i][0] == "E") & (i<(len(data)-1)) ):
        #def __initi__(self, success, time, user, nominator, votes):
        try:
            tempElection = Election(\
                               int(re.findall("\d+",data[i])[0]),
                               0, #time TODO
                               re.findall("\D+\d*",data[i+2])[1].strip(), #user
                               re.findall("\D+\d*",data[i+3])[1].strip(), #nominators can be "UNKNOWN"
                               ["Not yet added"] #votes, seprate object added a few cells down
                              )
            newTempElection= True
            
        except:
            #uncomment these lines for troubleshooting those 8 problem users
            #print("Problemm with :", data[i])
            #print(data[i+1])
            #print(data[i+2])
            #print(data[i+3])
            #print("----")
            continue
        if (newTempElection==True):
            electionsList.append(tempElection)
    else:   
        newTempElection = False

In [29]:
for i in range(1233,1245):
    print(electionsList[i].user +" nomitnated by:\t" + electionsList[i].nominator)

slyder_pilot nomitnated by:	UNKNOWN
pilotguy nomitnated by:	tawker
puckmv nomitnated by:	UNKNOWN
danielrocks123 nomitnated by:	UNKNOWN
sam_vimes nomitnated by:	aloan
dljessup nomitnated by:	UNKNOWN
herostratus nomitnated by:	tantalumtelluride
sean_gorter nomitnated by:	UNKNOWN
mtz206 nomitnated by:	hughcharlesparker
aaron_teh_tennisman nomitnated by:	UNKNOWN
jake0 nomitnated by:	UNKNOWN
ft2 nomitnated by:	UNKNOWN


In [30]:
len(electionsList)
#Expected length.  Success!
#The missing users have been fixed with better regexes and encoding

2793

<s>There's eight entries for users who have numbers as usernames or no username, just a user ID.
This could be cleaned up but for now, it'll just be discarded.</s>

In [31]:
for i in electionsList[2:14]:
    print(i.user +" nominated by:\t" + i.nominator)
#electionsList works as expected

andrevan nominated by:	node_ue
arminius nominated by:	thecustomoflife
lst27 nominated by:	UNKNOWN
chmod007 nominated by:	UNKNOWN
taoster nominated by:	UNKNOWN
anÃ¡rion nominated by:	UNKNOWN
jor nominated by:	UNKNOWN
proteus nominated by:	lord_emsworth
pumpie nominated by:	UNKNOWN
nichalp nominated by:	krs
pedanticallyspeaking nominated by:	UNKNOWN
benc nominated by:	neutrality


### Votes objects

In [32]:
votesList= [] #this will be a lists of lists.  Each sub-list will be addes to the election object
tempVotes=[]
votingBool = False

for i in data:
    if i [0]!= "V":
        if (len(tempVotes)>0):
            votesList.append(tempVotes)
        tempVotes=[]
    else:
        tempVotes.append(i)

In [33]:
votesList[-1][:5]
#the list of lists works as expected.  Next step is regex to clean it up

['V\t-1\t5800\t2008-01-05 23:31:00\tpedro\n',
 'V\t-1\t8045\t2008-01-05 23:34:00\tsarcasticidealist\n',
 'V\t-1\t7053\t2008-01-05 23:36:00\tavruch\n',
 'V\t-1\t6885\t2008-01-06 00:44:00\tkrator\n',
 'V\t-1\t8243\t2008-01-06 01:07:00\trazorflame\n']

In [34]:
(re.findall("\D+",votesList[0][0]))[-1].strip()

'neutrality'

In [35]:
len(votesList)
#The length of this list is the same length as the number of elections

2793

In [36]:
#The cell generates the list of listObjects, this cell singlehandedly took me 90 minutes
votesObjectList =[]
tempList =[]

for item1 in votesList:
    if len(tempList)>0:
        votesObjectList.append(tempList)
        tempList=[]
    for item2 in item1:
        # def __init__(self, vote, user_id, time, username):
        tempVote = Vote(\
                        int(re.findall("\d+",item2)[0]), #vote
                        re.findall("\d+",item2)[1], #user_id, possibly turn to int TODO
                        re.findall("\d+",item2)[2:], #time, in a list, TODO
                        re.findall("\D+\d*\D*", item2)[-1].strip() #username[2:]
                     )
        tempList.append(tempVote)

if len(tempList)>0:
        votesObjectList.append(tempList)


In [37]:
len(votesObjectList)
#length as expected, success

2793

In [38]:
print(electionsList[0].votes)

['Not yet added']


In [39]:
for i in range(len(electionsList)):
    electionsList[i].votes= votesObjectList[i]

In [40]:
for i in electionsList[:4]:
    print("Election for:", i.user)
    print(" - - - - -")
    for q in i.votes[:6]:
        print (q.user_name +\
               " voted " + \
               str(q.vote) + \
                ", userID: " \
                   + str(q.user_id)\
                  )
    print("_________")
    # someone has a screwy user name, I have to fix this TODO
    #note usernames can have numbers, need to fix regex

Election for: zoney
 - - - - -
neutrality voted 1, userID: 28
chmod007 voted 1, userID: 33
norm voted 1, userID: 34
orthogonal voted 1, userID: 5
michael voted 1, userID: 20
antandrus voted 1, userID: 36
_________
Election for: gerald_farinas
 - - - - -
neutrality voted 1, userID: 28
thecustomoflife voted 1, userID: 55
grunt voted 0, userID: 26
rhymeless voted 1, userID: 21
mirv voted 1, userID: 9
orthogonal voted 1, userID: 5
_________
Election for: andrevan
 - - - - -
node voted 1, userID: 62
slowking voted 1, userID: 27
merovingian voted 1, userID: 11
norm voted 1, userID: 34
anÃ¡rion voted 1, userID: 10
thecustomoflife voted 1, userID: 55
_________
Election for: arminius
 - - - - -
thecustomoflife voted 1, userID: 55
neutrality voted 1, userID: 28
antandrus voted 1, userID: 36
chmod007 voted 1, userID: 33
austin voted 0, userID: 51
ffirehorse voted 1, userID: 19
_________


#### Note: The above cell is good enough, by my judgement.  It's possible there is a better RegEx for usernames, and It's also possible that times as node and edge attributed could be collected.  I'm skipping that for now

### TODO:  Get votes list into the relevant elections, possibly using vote objects made above (vote object defined to better preserve attributes).

### TODO2: Once we have working list of the elections and votes, we can make the nodes (users from the votes, elections from the election objects.)  Then we can create a bipartate graph to begin Project 2 analysis

### Pandas
TODO

In [41]:
print(len([i.user for i in electionsList]))
print(len([i.nominator for i in electionsList]))

2793
2793


In [42]:
for j in [i.votes for i in electionsList[:1]]:
    for k in j[:3]:
        print(k.user_name)


neutrality
chmod007
norm


In [95]:
#this is all the user_name in all the votes.  gotten from nested list comprehension.  Example below:
#[[j for j in range(5)] for i in range(5)]


#[[j.user_name for j in election.votes]  for election in electionsList[:3]]

In [96]:
#this is all the votes in all the votes.  gotten from nested list comprehension.  1 is a yay

#[[j.vote for j in election.votes]  for election in electionsList[:2]]

In [56]:
df1 = pd.DataFrame({\
                   "Election" : [i.user for i in electionsList],
                    "Nominator" : [i.nominator for i in electionsList],
                    "Voters" : [[j.user_name for j in election.votes]  for election in electionsList],
                    "Votes" : [[j.vote for j in election.votes]  for election in electionsList] 
                   }
                  )
df1.head(10)# other attributes could be added

,Election,Nominator,Voters,Votes
0,zoney,neutrality,"[neutrality, chmod007, norm, orthogonal, micha...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,gerald_farinas,neutrality,"[neutrality, thecustomoflife, grunt, rhymeless...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ..."
2,andrevan,node_ue,"[node, slowking, merovingian, norm, anÃ¡rion, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,arminius,thecustomoflife,"[thecustomoflife, neutrality, antandrus, chmod...","[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, ..."
4,lst27,UNKNOWN,"[chmod007, anÃ¡rion, everyking, thecustomoflif...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, ..."
5,chmod007,UNKNOWN,"[merovingian, fredrik, kate, jwrosenzweig, mir...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,taoster,UNKNOWN,"[andrevan, grunt, merovingian, slowking, cyp, ...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ..."
7,anÃ¡rion,UNKNOWN,"[mirv, jor, gzornenplatz, neutrality, orthogon...","[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,jor,UNKNOWN,"[cecropia, radicalbender, chris, wik, danny, r...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..."
9,proteus,lord_emsworth,"[mirv, orthogonal, ffirehorse, ugen64, merovin...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Next Part: NetworkX

In [ ]:
from networkx.algorithms import bipartite

https://networkx.org/documentation/stable/reference/algorithms/bipartite.html

B = nx.Graph()

#### Add nodes with the node attribute "bipartite"

B.add_nodes_from([1, 2, 3, 4], bipartite=0)

B.add_nodes_from(["a", "b", "c"], bipartite=1)

#### Add edges only between nodes of opposite node sets

B.add_edges_from([(1, "a"), (1, "b"), (2, "b"), (2, "c"), (3, "c"), (4, "a")])

In [91]:
B = nx.Graph()

In [92]:
B.add_nodes_from([user.user for user in electionsList], bipartate=0) #users for elections
#in other words, who's being voted on
#TODO add nominator attribute

In [63]:
#TODO ????
#
allVotersList = [[j.user_name for j in election.votes]  for election in electionsList]
len(allVotersList)

2793

In [86]:
allVoters = []
for elect in allVotersList:
    for voter in elect:
        allVoters.append(voter)

In [87]:
print (len(allVoters))

114013


In [88]:
round( (len(allVoters)) / len(set (allVoters)) , 2)
#Each voter participates in 18.5 elections

18.5

6162


In [93]:
B.add_nodes_from(set(allVoters), bipartate=1) #All voters, second node type

In [94]:
B.nodes()

NodeView(('zoney', 'gerald_farinas', 'andrevan', 'arminius', 'lst27', 'chmod007', 'taoster', 'anÃ¡rion', 'jor', 'proteus', 'pumpie', 'nichalp', 'pedanticallyspeaking', 'benc', 'edge', 'golbez', 'chuq', 'kloy1334', 'postdlf', 'mackensen', 'cryptoderk', 'poccil', 'rdsmith4', 'matt_crypto', 'sam_spade', 'aloan', 'fire_star', 'neutrality', 'jallan', 'slowking_man', 'gwalla', 'hawstom', 'ta_bu_shi_da_yu', 'skyler1534', 'cyborgtosser', 'rhobite', 'ffirehorse', 'cool_hand_luke', 'norm', 'lucky_6', 'scudlee', 'spencer195', 'ludraman', 'anthony', 'marine_69', 'swadair', 'netoholic', 'aranel', 'violetriga', 'p3', 'duncharris', 'lachatdelarue', 'philhibbs', 'dbachmann', 'cogden', 'ran', 'hyacinth', 'df08', 'joy_stovall', 'ortolan88', 'dankashet', 'derek_ross', 'eequor', 'indrian', 'wapcaplet', 'karada', 'frecklefoot', 'aris_katsaris', 'jeronimo', 'clockworktroll', 'yelyos', 'gamaliel', 'shaneking', 'takuyamurata', 'maury_markowitz', 'kf', 'cstar', 'lgagnon', 'zoicon5', 'timc', 'brg', 'dmn', 'jpgo